<a href="https://colab.research.google.com/github/dave502/NLP/blob/main/lesson_09/nlp_hw_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
from pathlib import Path
from keras.callbacks import EarlyStopping

Загрузка данных

In [2]:
%%capture
!wget https://www.dropbox.com/s/wfrfgaizixxwz3w/evgenyi_onegin.txt

In [3]:
!ls

evgenyi_onegin.txt  sample_data


In [4]:
data_path = Path('evgenyi_onegin.txt')
text = data_path.read_text()
print(f'text lenght is {len(text)} chars')
print(f'text sample:\n {text[:500]}')

text lenght is 286984 chars
text sample:
 Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высо


Удвоение объёмаданных для лучшего обучения

In [5]:
text = text + text

Словарь всех символов текста

In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters: \n{np.array(vocab+[""]).reshape(4, -1)}')

131 unique characters: 
[['\n' ' ' '!' '"' "'" '(' ')' ',' '-' '.' '0' '1' '2' '3' '4' '5' '6'
  '7' '8' '9' ':' ';' '?' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'L']
 ['M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'V' 'W' 'X' 'Y' 'a' 'b' 'c' 'd' 'e' 'f'
  'g' 'h' 'i' 'k' 'l' 'm' 'n' 'o' 'p' 'q' 'r' 's' 't' 'u' 'v']
 ['w' 'y' 'z' '{' '}' 'А' 'Б' 'В' 'Г' 'Д' 'Е' 'Ж' 'З' 'И' 'К' 'Л' 'М' 'Н'
  'О' 'П' 'Р' 'С' 'Т' 'У' 'Ф' 'Х' 'Ц' 'Ч' 'Ш' 'Ь' 'Э' 'Ю' 'Я']
 ['а' 'б' 'в' 'г' 'д' 'е' 'ж' 'з' 'и' 'й' 'к' 'л' 'м' 'н' 'о' 'п' 'р' 'с'
  'т' 'у' 'ф' 'х' 'ц' 'ч' 'ш' 'щ' 'ъ' 'ы' 'ь' 'э' 'ю' 'я' '']]


Меппиниги для преобразования токенов в индексы и обратно

In [7]:
# мэппинги 'символ <-> индекс'
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

Преобразование текста в индексы

In [8]:
text_as_int = np.array([char2idx[c] for c in text])
list(zip(text_as_int, text))[:20]

[(71, 'А'),
 (110, 'л'),
 (104, 'е'),
 (109, 'к'),
 (116, 'с'),
 (99, 'а'),
 (112, 'н'),
 (103, 'д'),
 (115, 'р'),
 (1, ' '),
 (87, 'С'),
 (104, 'е'),
 (115, 'р'),
 (102, 'г'),
 (104, 'е'),
 (104, 'е'),
 (101, 'в'),
 (107, 'и'),
 (122, 'ч'),
 (1, ' ')]

Преобразование индексированного текста в тензор

In [11]:
# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
# проверка обратного преобразования тензора
[idx2char[i.numpy()] for i in char_dataset.take(10)]

['А', 'л', 'е', 'к', 'с', 'а', 'н', 'д', 'р', ' ']

Разбиение тензора на последовательности по 100 символов

In [12]:
# The maximum length sentence for a single input in characters
seq_length = 100

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
[idx2char[i.numpy()] for i in sequences.take(1)]

[array(['А', 'л', 'е', 'к', 'с', 'а', 'н', 'д', 'р', ' ', 'С', 'е', 'р',
        'г', 'е', 'е', 'в', 'и', 'ч', ' ', 'П', 'у', 'ш', 'к', 'и', 'н',
        '\n', '\n', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'Е', 'в', 'г', 'е', 'н',
        'и', 'й', ' ', 'О', 'н', 'е', 'г', 'и', 'н', '\n', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], dtype='<U1')]

Преобразование тензора в тензорный кортеж (признак, класс)

In [13]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [14]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '
Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


In [15]:
# Batch size
BATCH_SIZE = 64
# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000
EPOCHS = 20

Разделение датасета на батчи

In [16]:
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

Параметры сеток

In [17]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 1024

# loss function sparse_categorical_crossentropy
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

**Сетка с GRU слоями**

In [20]:
class GRUgenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, batch_size):
        super(GRUgenerator, self).__init__()
        
        self.emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
                                 
        self.gru1 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
        self.gru2 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
                           
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        emb_x = self.emb(x)
        x1 = self.gru1(emb_x)
        x = x1
        for _ in range(3):
            x = self.gru2(x)

        x = (x + x1)/2
        return self.fc(x)

model_gru = GRUgenerator(vocab_size, embedding_dim, BATCH_SIZE)

model_gru.compile(optimizer='adam', loss=loss)

In [21]:
history = model_gru.fit(dataset, epochs=EPOCHS)

Epoch 1/20
88/88 [==============================] - 29s 279ms/step - loss: 2.1260
Epoch 2/20
88/88 [==============================] - 25s 281ms/step - loss: 1.6924
Epoch 3/20
88/88 [==============================] - 24s 269ms/step - loss: 1.5266
Epoch 4/20
88/88 [==============================] - 26s 284ms/step - loss: 1.4245
Epoch 5/20
88/88 [==============================] - 25s 274ms/step - loss: 1.3500
Epoch 6/20
88/88 [==============================] - 25s 279ms/step - loss: 1.2887
Epoch 7/20
88/88 [==============================] - 25s 279ms/step - loss: 1.2419
Epoch 8/20
88/88 [==============================] - 24s 269ms/step - loss: 1.2162
Epoch 9/20
88/88 [==============================] - 25s 273ms/step - loss: 1.1736
Epoch 10/20
88/88 [==============================] - 26s 284ms/step - loss: 1.1403
Epoch 11/20
88/88 [==============================] - 24s 271ms/step - loss: 1.0899
Epoch 12/20
88/88 [==============================] - 24s 270ms/step - loss: 1.0558
Epoch 13/20
8

Функция получения предикта модели и преобразования результатов в текст

In [18]:
def generate_text(model, start_string, temperature=0.5):
    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.

    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

Результат предикта GRU модели

In [24]:
text_ = generate_text(model_gru, start_string=u"И вот идет уже ",  temperature=0.7)
print(text_)

И вот идет уже дизвый         X
                                                 К                                                    киворадивитедогогомнедвой                                                  наде                                                                                         Поплечиреласа   селе                                           тотый.
                          Вста                                                       Ной                                                       


*Результат получился неудовлетворительный, изменение температуры не дало желаемого результата. При увеличении температуры сетка выводит произвольные символы, при уменьшении - увеличивается число пробелов*

**Сетка с LSTM слоями**

In [58]:
def build_lstm_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [94]:
model_lstm = build_lstm_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [81]:
model_lstm.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (64, None, 128)           16768     
                                                                 
 lstm_28 (LSTM)              (64, None, 1024)          4722688   
                                                                 
 lstm_29 (LSTM)              (64, None, 1024)          8392704   
                                                                 
 lstm_30 (LSTM)              (64, None, 1024)          8392704   
                                                                 
 lstm_31 (LSTM)              (64, None, 1024)          8392704   
                                                                 
 dense_7 (Dense)             (64, None, 131)           134275    
                                                                 
Total params: 30,051,843
Trainable params: 30,051,843


In [95]:
model_lstm.compile(optimizer='adam', loss=loss)

In [83]:
!rm -rf ./training_checkpoints

In [84]:
!ls ./training_checkpoints

ls: cannot access './training_checkpoints': No such file or directory


In [96]:
# Directory where the checkpoints will be saved
checkpoint_dir = Path('./training_checkpoints')
# Name of the checkpoint files
checkpoint_prefix = checkpoint_dir / "ckpt_{epoch}"

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*3,
    save_weights_only=True)

In [86]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model_lstm(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 131) # (batch_size, sequence_length, vocab_size)


In [87]:
history = model_lstm.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
88/88 [==============================] - 37s 361ms/step - loss: 2.4216
Epoch 2/20
88/88 [==============================] - 31s 344ms/step - loss: 2.1502
Epoch 3/20
88/88 [==============================] - 33s 365ms/step - loss: 1.7189
Epoch 4/20
88/88 [==============================] - 32s 351ms/step - loss: 1.5331
Epoch 5/20
88/88 [==============================] - 32s 351ms/step - loss: 1.4367
Epoch 6/20
88/88 [==============================] - 32s 362ms/step - loss: 1.3847
Epoch 7/20
88/88 [==============================] - 32s 352ms/step - loss: 1.3371
Epoch 8/20
88/88 [==============================] - 32s 353ms/step - loss: 1.3111
Epoch 9/20
88/88 [==============================] - 33s 363ms/step - loss: 1.2915
Epoch 10/20
88/88 [==============================] - 32s 351ms/step - loss: 1.2858
Epoch 11/20
88/88 [==============================] - 32s 351ms/step - loss: 1.2469
Epoch 12/20
88/88 [==============================] - 33s 366ms/step - loss: 1.2258
Epoch 13/20
8

In [88]:
tf.train.latest_checkpoint(checkpoint_dir)
model_lstm = build_lstm_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model_lstm.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model_lstm.build(tf.TensorShape([1, None]))

In [89]:
model_lstm.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (1, None, 128)            16768     
                                                                 
 lstm_32 (LSTM)              (1, None, 1024)           4722688   
                                                                 
 lstm_33 (LSTM)              (1, None, 1024)           8392704   
                                                                 
 lstm_34 (LSTM)              (1, None, 1024)           8392704   
                                                                 
 lstm_35 (LSTM)              (1, None, 1024)           8392704   
                                                                 
 dense_8 (Dense)             (1, None, 131)            134275    
                                                                 
Total params: 30,051,843
Trainable params: 30,051,843


Проверка результата

In [91]:
text_ = generate_text(model_lstm, start_string=u"И вот идет уже ",  temperature=0.7)
print(text_)

И вот идет уже лизна гило:
                        Мого поремненной избови
                        Свиренскутельных усталась,
                        По стам и внавдась не подоле!
                        И зна мечтих на венский гранила,
                        В страгой нашди невовники,
                        Попленин                                                                                                                 XXXVI

                        То он пиривно из подоне
                        Пор


*Результат далёк от идеального, но получше чем у сетки с GRU слоями*